In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import re
import os

# Завантажуємо датасет з правильним кодуванням
df = pd.read_csv('book_details.csv', encoding='latin-1')

print("=== ЗАВАНТАЖЕННЯ ДАТАСЕТУ ===")
print(f"Загальна кількість книг: {len(df)}")
print(f"Колонки: {df.columns.tolist()}")
print(f"Розмір: {df.shape}")

# Показуємо перші записи
df.head()

=== ЗАВАНТАЖЕННЯ ДАТАСЕТУ ===
Загальна кількість книг: 13324
Колонки: ['title', 'author', 'rating', 'no_of_ratings', 'no_of_reviews', 'description', 'genres']
Розмір: (13324, 7)


,title,author,rating,no_of_ratings,no_of_reviews,description,genres
0,Divergent,Veronica Roth,4.15,3765886,"117,791","In Beatrice Prior's dystopian Chicago world, s...","Young Adult, Dystopia, Fantasy, Fiction, Scien..."
1,Catching Fire,Suzanne Collins,4.31,3305054,"113,480",Sparks are igniting.Flames are spreading.And t...,"Young Adult, Dystopia, Fiction, Fantasy, Scien..."
2,The Fault in Our Stars,John Green,4.15,4851513,"174,662",Despite the tumor-shrinking medical miracle th...,"Young Adult, Romance, Fiction, Contemporary, R..."
3,To Kill a Mockingbird,Harper Lee,4.27,5784553,"112,055",The unforgettable novel of a childhood in a sl...,"Classics, Fiction, Historical Fiction, School,..."
4,The Lightning Thief,Rick Riordan,4.30,2752945,"87,446",Alternate cover for this ISBN can be found her...,"Fantasy, Young Adult, Mythology, Fiction, Midd..."


In [10]:
print("=== АНАЛІЗ ДАНИХ ===")
print(f"Розмір датасету: {df.shape}")
print("\nПорожні значення по колонкам:")
print(df.isnull().sum())

print("\nПриклади значень:")
print(f"Author: {df['author'].head(3).tolist()}")
print(f"Genres: {df['genres'].head(3).tolist()}")
print(f"Description length: {[len(str(x)) for x in df['description'].head(3).tolist()]}")

# Перевіряємо проблемні символи
print(f"\nКількість записів з символами � в описі: {df['description'].str.contains('�', na=False).sum()}")
print(f"Кількість записів з символами � в жанрах: {df['genres'].str.contains('�', na=False).sum()}")
print(f"Кількість записів з символами � в авторах: {df['author'].str.contains('�', na=False).sum()}")

=== АНАЛІЗ ДАНИХ ===
Розмір датасету: (13324, 7)

Порожні значення по колонкам:
title              0
author             0
rating             0
no_of_ratings      0
no_of_reviews      0
description       51
genres           997
dtype: int64

Приклади значень:
Author: ['Veronica Roth', 'Suzanne Collins', 'John Green']
Genres: ['Young Adult, Dystopia, Fantasy, Fiction, Science Fiction, Romance, Adventure', 'Young Adult, Dystopia, Fiction, Fantasy, Science Fiction, Romance, Adventure', 'Young Adult, Romance, Fiction, Contemporary, Realistic Fiction, Teen, Coming Of Age']
Description length: [1400, 1205, 547]

Кількість записів з символами � в описі: 0
Кількість записів з символами � в жанрах: 0
Кількість записів з символами � в авторах: 0


In [11]:
print("=== ОЧИСТКА ДАНИХ ===")

# Залишаємо тільки потрібні колонки
df_clean = df[['title', 'author', 'genres', 'description']].copy()

print(f"Початкова кількість записів: {len(df_clean)}")

# Очищуємо від символів � та інших проблемних символів
df_clean['author'] = df_clean['author'].fillna('').astype(str).str.replace('�', '', regex=False)
df_clean['genres'] = df_clean['genres'].fillna('').astype(str).str.replace('�', '', regex=False)
df_clean['description'] = df_clean['description'].fillna('').astype(str).str.replace('�', '', regex=False)

# Видаляємо записи де всі три поля порожні
mask = (df_clean['author'].str.strip() != '') | (df_clean['genres'].str.strip() != '') | (df_clean['description'].str.strip() != '')
df_clean = df_clean[mask].copy()

print(f"Після видалення порожніх записів: {len(df_clean)}")

# Формуємо комбінований текст для TF-IDF (БЕЗ назви книги!)
df_clean['combined_text'] = (
    df_clean['author'].fillna('').astype(str) + ' ' + 
    df_clean['genres'].fillna('').astype(str) + ' ' + 
    df_clean['description'].fillna('').astype(str)
).str.strip()

# Видаляємо записи з порожнім combined_text
df_clean = df_clean[df_clean['combined_text'].str.strip() != ''].copy()

print(f"Після очистки залишилось: {len(df_clean)} книг")
print(f"\nПриклад комбінованого тексту:")
print(f"Назва: {df_clean.iloc[0]['title']}")
print(f"Текст для векторизації: {df_clean.iloc[0]['combined_text'][:200]}...")

=== ОЧИСТКА ДАНИХ ===
Початкова кількість записів: 13324
Після видалення порожніх записів: 13324
Після очистки залишилось: 13324 книг

Приклад комбінованого тексту:
Назва: Divergent
Текст для векторизації: Veronica Roth Young Adult, Dystopia, Fantasy, Fiction, Science Fiction, Romance, Adventure In Beatrice Prior's dystopian Chicago world, society is divided into five factions, each dedicated to the cul...


In [13]:
print("=== НАВЧАННЯ TF-IDF МОДЕЛІ ===")

# Налаштування векторизатора
vectorizer = TfidfVectorizer(
    max_features=5000,        # Максимум 5000 слів
    min_df=2,                # Слово має бути мінімум у 2 документах
    max_df=0.8,              # Максимум у 80% документів
    ngram_range=(1, 2),      # Юніграми та біграми
    stop_words='english',    # Англійські стоп-слова
    lowercase=True,
    strip_accents='unicode'
)

# Готуємо корпус
corpus = df_clean['combined_text'].tolist()
print(f"Кількість документів для навчання: {len(corpus)}")

# Навчаємо векторизатор
tfidf_matrix = vectorizer.fit_transform(corpus)

print(f"Форма TF-IDF матриці: {tfidf_matrix.shape}")
print(f"Кількість унікальних слів у словнику: {len(vectorizer.get_feature_names_out())}")

# Перевіряємо щільність матриці
density = (tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1])) * 100
print(f"Щільність матриці: {density:.2f}%")
print(f"Загальна кількість ненульових елементів: {tfidf_matrix.nnz}")

=== НАВЧАННЯ TF-IDF МОДЕЛІ ===
Кількість документів для навчання: 13324
Форма TF-IDF матриці: (13324, 5000)
Кількість унікальних слів у словнику: 5000
Щільність матриці: 1.28%
Загальна кількість ненульових елементів: 855559


In [15]:
print("=== ЗБЕРЕЖЕННЯ МОДЕЛІ ===")

# Зберігаємо в папку з датасетом
model_dir = '/home/stanislav/diplom/recommendations/'

# Зберігаємо векторизатор
vectorizer_path = os.path.join(model_dir, 'tfidf_vectorizer.pkl')
with open(vectorizer_path, 'wb') as f:
    pickle.dump(vectorizer, f)

print(f"✅ Векторизатор збережено: {vectorizer_path}")

# Перевіряємо збереження
vectorizer_size = os.path.getsize(vectorizer_path) / 1024  # KB
print(f"Розмір файлу векторизатора: {vectorizer_size:.1f} KB")

# Показуємо структуру папки
print(f"\nФайли в папці {model_dir}:")
for file in os.listdir(model_dir):
    if file.endswith(('.pkl', '.csv', '.ipynb')):
        print(f"- {file}")

=== ЗБЕРЕЖЕННЯ МОДЕЛІ ===
✅ Векторизатор збережено: /home/stanislav/diplom/recommendations/tfidf_vectorizer.pkl
Розмір файлу векторизатора: 182.7 KB

Файли в папці /home/stanislav/diplom/recommendations/:
- book_details.csv
- ContentBased.ipynb
- tfidf_vectorizer.pkl


In [19]:
def get_recommendations(book_indices, top_k=8):
    """
    Отримати рекомендації на основі списку індексів книг
    """
    if not book_indices or len(book_indices) == 0:
        print("Немає переглянутих книг для рекомендацій")
        return [], []
    
    print(f"Обчислення рекомендацій на основі {len(book_indices)} книг...")
    
    # Отримуємо вектори обраних книг
    user_vectors = tfidf_matrix[book_indices]
    
    # Обчислюємо середній профіль користувача
    user_profile = user_vectors.mean(axis=0)
    
    # ПОВНЕ ВИПРАВЛЕННЯ: Перетворюємо sparse матриці в numpy arrays
    if hasattr(user_profile, 'toarray'):
        user_profile = user_profile.toarray()
    
    # Перетворюємо tfidf_matrix в dense format для косинусної подібності
    tfidf_dense = tfidf_matrix.toarray()
    
    # Обчислюємо косинусну подібність
    similarities = cosine_similarity(user_profile, tfidf_dense).ravel()
    
    # Виключаємо вже переглянуті книги
    for idx in book_indices:
        similarities[idx] = -1
    
    # Знаходимо топ рекомендації
    top_indices = similarities.argsort()[-top_k:][::-1]
    top_similarities = similarities[top_indices]
    
    return top_indices, top_similarities

print("✅ Функція рекомендацій повністю виправлена!")

✅ Функція рекомендацій повністю виправлена!


In [20]:
print("=== ТЕСТУВАННЯ РЕКОМЕНДАЦІЙ ===")

# Випадковий тест
viewed_books = [0, 5, 10]

print("Переглянуті книги:")
for i, idx in enumerate(viewed_books):
    book = df_clean.iloc[idx]
    print(f"{i+1}. {book['title']} by {book['author']}")
    print(f"   Жанри: {book['genres']}")

# Отримуємо рекомендації
rec_indices, similarities = get_recommendations(viewed_books, top_k=8)

print(f"\nРекомендації (топ-8):")
for i, (idx, sim) in enumerate(zip(rec_indices, similarities)):
    book = df_clean.iloc[idx]
    print(f"{i+1}. {book['title']} by {book['author']} (подібність: {sim:.3f})")
    print(f"   Жанри: {book['genres']}")
    print()

=== ТЕСТУВАННЯ РЕКОМЕНДАЦІЙ ===
Переглянуті книги:
1. Divergent by Veronica Roth
   Жанри: Young Adult, Dystopia, Fantasy, Fiction, Science Fiction, Romance, Adventure
2. Harry Potter and the Prisoner of Azkaban by J.K. Rowling
   Жанри: Fantasy, Fiction, Young Adult, Magic, Childrens, Middle Grade, Audiobook
3. Pride and Prejudice by Jane Austen
   Жанри: Classics, Fiction, Romance, Historical Fiction, Literature, Historical, Audiobook
Обчислення рекомендацій на основі 3 книг...


TypeError: np.matrix is not supported. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html